# Data Preprocessing

In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
import pickle
from random import shuffle
import matplotlib.pyplot as plt
import datetime

C:\Users\kabil\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
dataset = os.path.join(os.getcwd(), "pdataset")
data = []
for i , value in enumerate(os.listdir(dataset)):
    f_dir = os.path.join(dataset,value)
    for each in os.listdir(f_dir):
        image = cv2.imread(os.path.join(f_dir,each),0)
        image = cv2.resize(image,(200,200))
        data.append((image,i))

In [ ]:
shuffle(data)
x = []
y = []
for features , label in data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(-1,200,200,1)
pickle_out = open("X.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

# Training

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  

model.add(Dense(64))
model.add(Activation('sigmoid'))


model.add(Dense(1))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

logdir = "logs\\scalars\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3,  callbacks=[tensorboard_callback])

model.save("model.h5")

# Testing

In [5]:
model = tf.keras.models.load_model('model.h5')
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 198, 198, 64)      640       
_________________________________________________________________
activation_3 (Activation)    (None, 198, 198, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 97, 97, 64)        36928     
_________________________________________________________________
activation_4 (Activation)    (None, 97, 97, 64)        0         
________________

In [ ]:
def test(path):
    image = cv2.imread(path,1)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,(5,5),0)
    min_x = image.shape[0]
    min_y = image.shape[1]
    max_x = 0
    max_y = 0
    canny = cv2.Canny(blurred,0,250)
    (cnts,_) = cv2.findContours(canny.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    coints = image.copy()
    frame = image.copy()
    cv2.drawContours(frame,cnts,-1,(255,0,0),2)
    for cnt in cnts:
        x ,y,w,h = cv2.boundingRect(cnt)
        area = w * h
        if area > ((image.shape[0] * image.shape[1]) * 0.2):
            if min_x > x:
                min_x = x
            if min_y > y:
                min_y = y
            if max_x < (x + w):
                max_x = x + w
            if max_y < (y + h):
                max_y = y +h
    cv2.rectangle(coints,(min_x,min_y),(max_x,max_y),(0,255,0),2)
    roi = gray[min_y:max_y,min_x:max_x]
    image = cv2.resize(roi,(200,200))
    x = np.array(image).reshape(-1,200,200,1)
    y = model.predict(x)
    print(y[0][0])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def ptest(path):
    image = cv2.imread(path,0)
    plt.imshow(image)
    plt.show()
    image = cv2.resize(image,(200,200))
    x = np.array(image).reshape(-1,200,200,1)
    y = model.predict(x)
    print(y[0][0])

In [ ]:
for i , each in enumerate(os.listdir("ptest")):
    print(each,f"expected {i} : observerd ->")

    for file in os.listdir(f"ptest/{each}"):
        print(file)
        
        test(f"ptest/{each}/"+file)


In [ ]:
for layer in model.layers:
        weights, bias= layer.get_weights()
        
        f_min, f_max = weights.min(), weights.max()
        
        filters = (weights - f_min) / (f_max - f_min)  
        print(layer.name, filters.shape)

        for i in range(filters.shape[3]):
            filt=filters[:,:,:, i]
            print(filt)
          

In [ ]:
for layer in model.layers:
      if 'conv' in layer.name:
        weights, bias= layer.get_weights()
        
        f_min, f_max = weights.min(), weights.max()
        
        filters = (weights - f_min) / (f_max - f_min)  
        print(layer.name, filters.shape)

        print(filters.shape[3])
        filter_cnt=1
        
        for i in range(filters.shape[3]):
            filt=filters[:,:,:, i]
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:, j])
                filter_cnt+=1
        plt.show()

In [ ]:
!tensorboard --logdir logs/scalars